In [1]:
from time import sleep
import numpy as np
from os.path import dirname, join, isfile, isdir
from os import makedirs
from sys import exit
import pandas as pd
from Levenshtein import ratio
from epiweeks import Week
from datetime import datetime, timedelta

from bulletin.commom import static, normalize
from bulletin.data.sivep import sivep
from bulletin.data.sim import sim

pd.set_option('display.max_columns', None)

In [2]:
sivep = sivep()
sivep.read_all_database_files()

In [3]:
sivep_df = sivep.get_df()

In [4]:
sivep_df = sivep_df.loc[sivep_df['SG_UF'] == 'PR']
sivep_df = sivep_df.loc[sivep_df['SG_UF_NOT'] == 'PR']
sivep_df = sivep_df.loc[sivep_df['AMOSTRA'] == '1']
sivep_df.loc[sivep_df['CLASSI_FIN'].isnull(), 'CLASSI_FIN'] = '0'

In [5]:
sivep_df['DT_NOTIFIC'] = sivep_df['DT_NOTIFIC'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
sivep_df['SE'] = sivep_df['DT_NOTIFIC'].apply(lambda x: Week.fromdate(x))

In [6]:
tabela = sivep_df.groupby(['SE','CLASSI_FIN'])[['NU_NOTIFIC']].count().unstack()

In [7]:
tabela = tabela.droplevel(0, axis=1)

In [8]:
tabela['Semana Epidemiológica'] = [ x.week for x in tabela.index.values]
tabela = tabela.set_index('Semana Epidemiológica')

In [9]:
tabela['1'] = tabela['1'].add(tabela['2'])
tabela['1'] = tabela['3']
tabela = tabela[['5', '1', '4', '0']]

In [10]:
tabela = tabela.rename(columns={'5':'Casos confirmados', '1':'Casos descartados', '4':'Casos suspeitos', '0':'Não informados'})

In [11]:
tabela['Total testes'] = tabela.sum(1)
tabela['Positividade (%)'] = tabela['Casos confirmados'].div(tabela['Total testes'])

In [12]:
tabela.to_excel('output/exames_sivep.xlsx', index = False)

In [13]:
tabela

CLASSI_FIN,Casos confirmados,Casos descartados,Casos suspeitos,Não informados,Total testes,Positividade (%)
Semana Epidemiológica,,,,,,
1,1.0,NaN,8.0,NaN,9.0,0.111111
2,1.0,NaN,27.0,2.0,30.0,0.033333
3,1.0,NaN,26.0,NaN,27.0,0.037037
4,NaN,NaN,23.0,NaN,23.0,NaN
5,NaN,1.0,32.0,NaN,33.0,NaN
...,...,...,...,...,...,...
13,2306.0,3.0,319.0,830.0,3458.0,0.666859
14,1904.0,1.0,305.0,914.0,3124.0,0.609475
15,1560.0,NaN,230.0,1216.0,3006.0,0.518962
